# Import libraries and helper functions

Based on https://github.com/jacobgil/keras-grad-cam

**Note**: Requires Keras <= 2.0.8 (!)

In [1]:
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.layers.core import Lambda
from keras.models import Sequential
from tensorflow.python.framework import ops
import keras.backend as K
import tensorflow as tf
import numpy as np
import keras
import sys
import cv2 

def target_category_loss(x, category_index, nb_classes):
    return tf.multiply(x, K.one_hot([category_index], nb_classes))

def target_category_loss_output_shape(input_shape):
    return input_shape

def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)

def load_image(path):
    img_path = path
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x, data_format='channels_last')
    return x

def register_gradient():
    if "GuidedBackProp" not in ops._gradient_registry._registry:
        @ops.RegisterGradient("GuidedBackProp")
        def _GuidedBackProp(op, grad):
            dtype = op.inputs[0].dtype
            return grad * tf.cast(grad > 0., dtype) * \
                tf.cast(op.inputs[0] > 0., dtype)

def compile_saliency_function(model, activation_layer='block5_conv3'):
    input_img = model.input
    layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])
    layer_output = layer_dict[activation_layer].output
    max_output = K.max(layer_output, axis=3)
    saliency = K.gradients(K.sum(max_output), input_img)[0]
    return K.function([input_img, K.learning_phase()], [saliency])

def modify_backprop(model, name):
    g = tf.get_default_graph()
    with g.gradient_override_map({'Relu': name}):

        # get layers that have an activation
        layer_dict = [layer for layer in model.layers[1:]
                      if hasattr(layer, 'activation')]

        # replace relu activation
        for layer in layer_dict:
            if layer.activation == keras.activations.relu:
                layer.activation = tf.nn.relu

        # re-instanciate a new model
        new_model = VGG16(weights='imagenet')
    return new_model

def deprocess_image(x):
    '''
    Same normalization as in:
    https://github.com/fchollet/keras/blob/master/examples/conv_filter_visualization.py
    '''
    if np.ndim(x) > 3:
        x = np.squeeze(x)
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_dim_ordering() == 'th':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

def grad_cam(input_model, image, category_index, layer_name, num_classes, multi=False):
    model = Sequential()
    model.add(input_model)

    nb_classes = num_classes
    target_layer = lambda x: target_category_loss(x, category_index, nb_classes)
    model.add(Lambda(target_layer,
                     output_shape = target_category_loss_output_shape))

    loss = K.sum(model.layers[-1].output)
    if multi:
        conv_output =  [l for l in model.layers[0].layers if l.name is layer_name][0].get_output_at(1)
    else:
        conv_output =  [l for l in model.layers[0].layers if l.name is layer_name][0].output
    
    grads = normalize(K.gradients(loss, conv_output)[0])
    gradient_function = K.function([model.layers[0].input], [conv_output, grads])
    output, grads_val = gradient_function([image])
    output, grads_val = output[0, :], grads_val[0, :, :, :]

    weights = np.mean(grads_val, axis = (0, 1))
    cam = np.ones(output.shape[0 : 2], dtype = np.float32)

    for i, w in enumerate(weights):
        cam += w * output[:, :, i]

    cam = cv2.resize(cam, (224, 224))
    cam = np.maximum(cam, 0)
    heatmap = cam / np.max(cam)

    #Return to BGR [0..255] from the preprocessed image
    image = image[0, :]
    image -= np.min(image)
    image = np.minimum(image, 255)

    cam = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)
    cam = np.float32(cam) + np.float32(image)
    cam = 255 * cam / np.max(cam)
    return np.uint8(cam), heatmap


Using TensorFlow backend.


# Test on VGG model with ImageNet weights

In [2]:
vgg = VGG16(weights='imagenet')

for img in ["img_1", "img_336"]:
    preprocessed_input = load_image(img + ".png")

    predictions = vgg.predict(preprocessed_input)
#     top_1 = decode_predictions(predictions)[0][0]
#     print('Predicted class:')
#     print('%s (%s) with probability %.2f' % (top_1[1], top_1[0], top_1[2]))

    predicted_class = np.argmax(predictions)
    cam, heatmap = grad_cam(vgg, preprocessed_input, predicted_class, "block5_conv3", 1000)
    cv2.imwrite("gcam_" + img + "_VGG.jpg", cam)

    register_gradient()
    guided_model = modify_backprop(vgg, 'GuidedBackProp')
    saliency_fn = compile_saliency_function(guided_model)
    saliency = saliency_fn([preprocessed_input, 0])
    gradcam = saliency[0] * heatmap[..., np.newaxis]
    cv2.imwrite("guided_gcam_" + img + "_VGG.jpg", deprocess_image(gradcam))

# Test on VGG model with our own weights 

Remember to disable dropoff!

In [3]:
import keras.initializers as initializers
from keras.layers import Dense, Dropout, Flatten
from keras import regularizers
from keras.optimizers import Adam

num_dense = 4096
num_classes = 5

def build_model(init_seed=None):
    # CONVOLUTIONAL LAYERS: use the pretrained weights from ImageNet
    base_model = VGG16(weights='imagenet', input_shape=(224, 224, 3), include_top=False)
    
    # TOP LAYERS: Trainable dense layers with random initialization
    dense_model = Sequential()
    # Input layer
    dense_model.add(Flatten(input_shape=base_model.output_shape[1:], name="Flatten"))
    # FC6
    dense_model.add(Dense(num_dense, activation='relu', kernel_initializer=initializers.glorot_normal(seed=init_seed), kernel_regularizer=regularizers.l2(0.01), name="Dense1"))
    # FC6 Dropout
#     dense_model.add(Dropout(rate=0.5, name="Dropout1"))
    # FC7
    dense_model.add(Dense(num_dense, activation='relu', kernel_initializer=initializers.glorot_normal(seed=init_seed), kernel_regularizer=regularizers.l2(0.01), name="Dense2"))
    # FC7 Dropout
#     dense_model.add(Dropout(rate=0.5, name="Dropout2"))
    # Softmax
    dense_model.add(Dense(num_classes, activation='softmax', kernel_initializer=initializers.glorot_normal(seed=init_seed), kernel_regularizer=regularizers.l2(0.01), name="Dense3"))

    # CREATE THE FULL MODEL (stack the dense layers on the convolutional layers)
    model = Sequential()
    # Add convolutional layers
    for layer in base_model.layers:
        model.add(layer)
    # Fix weights in the convolutional layers
    for layer in model.layers:
        layer.trainable = False
    # Add fully conncected layers
    for layer in dense_model.layers:
        model.add(layer)
    
    
    # COMPILE THE MODEL
    # Optimizer (can't figure out what learning rate Albert used)
    adam = Adam(lr = 0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model.compile(loss='categorical_crossentropy',
                  optimizer=adam,
                  metrics=['accuracy'])

    return model


cnn = build_model(3)
cnn.load_weights('weights_fold0.hdf5')

In [4]:
for img in ["img_1", "img_336"]:
    preprocessed_input = load_image(img+".png")

    predictions = cnn.predict(preprocessed_input)
    predicted_class = np.argmax(predictions)

    cam, heatmap = grad_cam(cnn, preprocessed_input, predicted_class, "block5_conv3", 5, multi=True)
    cv2.imwrite("gcam_" + img + ".jpg", cam)

    register_gradient()
    guided_model = modify_backprop(cnn, 'GuidedBackProp')
    saliency_fn = compile_saliency_function(guided_model)
    saliency = saliency_fn([preprocessed_input, 0])
    gradcam = saliency[0] * heatmap[..., np.newaxis]
    cv2.imwrite("guided_gcam_" + img + ".jpg", deprocess_image(gradcam))

# TODO

- find way to visualize LCRN model
- loop over all images for one night with cnn.predict, get predicted_class, output to predctions list
- plot time vs predictions list and compare with time vs true labels